In [1]:
class FDA:
    def __init__(self,Data):
        self.Data = Data
        self.labels = Data['class_label'].copy()
        self.coef_ = None
        self.transformed_data = None
        self.S_W = None
              

    def fit_transform(self):
        #Calculating total within-class scatter
        X = self.Data
        y = self.labels
        
        labels_list = []
        
        # Seprating the labels from the dataframe
        
        for i in np.unique(self.labels):
            labels_list.append(X[X['class_label'] == i].values)
          
        # Calculating between class scatter
        temp_scatter = np.zeros((X.shape[1]-1, X.shape[1]-1))
        for i in labels_list:
            i = np.delete(i, i.shape[1]-1,axis=1)
            mean = np.mean(i,axis=0)
            for j in i:
                diff = (j-mean).reshape(X.shape[1]-1,1)
                temp_scatter += diff.dot(diff.T)
        
        self.S_W = temp_scatter
        
        temp_bt_scatter = np.zeros((X.shape[1]-1, X.shape[1]-1))
        
        X = X.drop('class_label', axis=1)
        
        global_mean = np.mean(X,axis = 0)
        global_mean = np.array(global_mean)
        for i in labels_list:
            i = np.delete(i, i.shape[1]-1,axis=1)
            mean = np.array(np.mean(i,axis=0))
            diff = (mean - global_mean).reshape(X.shape[1],1)
            temp_bt_scatter += (i.shape[0])*(diff.dot(diff.T))

        SW_inv = np.linalg.inv(self.S_W)
        
        product = np.dot(SW_inv, temp_bt_scatter)
        
        eigenvalues, eigenvectors = np.linalg.eig(product)
        eigenvalues = np.real(eigenvalues)
        max_index = np.argmax(eigenvalues)
        
        max_eigenvector = eigenvectors[:,max_index]
        
        self.coef_ = max_eigenvector
        
        transform_data = np.dot(X,self.coef_.reshape(X.shape[1],1))
        
        return transform_data
    

In [2]:
import numpy as np
class LogisticRegression():
    def __init__(self,LearningRate,iterations, Data, labels):
        self.alpha = LearningRate
        self.iterations = iterations
        self.Data = Data
        self.Label =labels
        self.W = np.zeros(self.Data.shape[1])
        self.m = 0
    def fit(self):
        for i in range(self.iterations):
            A = 1/(1+np.exp(-self.Data.dot(self.W) + self.m))
            
            temp = A - self.Label.T
            dW = np.dot(self.Data.T, temp)/self.m
            db = np.sum(temp)/self.m
            
            self.W = self.W - self.alpha*dW
            self.m = self.m - self.alpha*db
            
    def predict(self, X):
        final_prediction = 1/(1+np.exp(-(X.dot(self.W)) +self.m))    
        for i in range(0,len(final_prediction)):
            if final_prediction[i] < 0.5:
                final_prediction[i] = 0
            else:
                final_prediction[i] = 1
        return final_prediction

# Load the data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataset = pd.read_csv('Heart.csv')
dataset = dataset.drop('Unnamed: 0',axis=1)
dataset.head()

dataset = dataset.dropna()

In [4]:
dataset.describe()

,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca
count,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000
mean,54.542088,0.676768,131.693603,247.350168,0.144781,0.996633,149.599327,0.326599,1.055556,1.602694,0.676768
std,9.049736,0.468500,17.762806,51.997583,0.352474,0.994914,22.941562,0.469761,1.166123,0.618187,0.938965
min,29.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000
25%,48.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.000000,0.000000,0.000000,1.000000,0.000000
50%,56.000000,1.000000,130.000000,243.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000
75%,61.000000,1.000000,140.000000,276.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000
max,77.000000,1.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000


In [5]:
dataset.head(100)

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,59,1,asymptomatic,110,239,0,2,142,1,1.2,2,1.0,reversable,Yes
97,60,0,asymptomatic,150,258,0,2,157,0,2.6,2,2.0,reversable,Yes
98,52,1,nontypical,134,201,0,0,158,0,0.8,1,1.0,normal,No
99,48,1,asymptomatic,122,222,0,2,186,0,0.0,1,0.0,normal,No


# Make some changes in the AHD, Thal, Chest pain


In [6]:
from sklearn.linear_model import LogisticRegression
dataset['ChestPain'] = dataset['ChestPain'].replace({'typical' : 1, 'asymptomatic': 2, 'nonanginal':3, 'nontypical':4})
dataset['Thal'] = dataset['Thal'].replace({'fixed':1, 'normal':2, 'reversable':3, 'nan':4})
dataset['AHD'] = dataset['AHD'].replace({'No':0, 'Yes':1})

labels = dataset['AHD'].copy()

In [7]:
dataset.describe()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
count,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000
mean,54.542088,0.676768,2.531987,131.693603,247.350168,0.144781,0.996633,149.599327,0.326599,1.055556,1.602694,0.676768,2.326599,0.461279
std,9.049736,0.468500,0.858082,17.762806,51.997583,0.352474,0.994914,22.941562,0.469761,1.166123,0.618187,0.938965,0.585061,0.499340
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,48.000000,0.000000,2.000000,120.000000,211.000000,0.000000,0.000000,133.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,2.000000,130.000000,243.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,2.000000,0.000000
75%,61.000000,1.000000,3.000000,140.000000,276.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,3.000000,1.000000


In [8]:
dataset = dataset.rename(columns={'AHD':'class_label'})
dataset

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,class_label
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,1,0
1,67,1,2,160,286,0,2,108,1,1.5,2,3.0,2,1
2,67,1,2,120,229,0,2,129,1,2.6,2,2.0,3,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,2,0
4,41,0,4,130,204,0,2,172,0,1.4,1,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,2,140,241,0,0,123,1,0.2,2,0.0,3,1
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,3,1
299,68,1,2,144,193,1,0,141,0,3.4,2,2.0,3,1
300,57,1,2,130,131,0,0,115,1,1.2,2,1.0,3,1


# First apply FDA only

PreProcessing

In [9]:
preModel = FDA(dataset)
transformed_data =preModel.fit_transform()

In [10]:
transformed_data = np.real(transformed_data)

In [11]:
transformed_data

array([[-7.95588977e-01],
       [-3.44227804e+00],
       [-3.00123800e+00],
       [-8.84940904e-01],
       [ 1.07157806e-01],
       [-8.19146381e-03],
       [-1.78810308e+00],
       [-5.00280510e-01],
       [-1.88613825e+00],
       [-2.01243603e+00],
       [-3.82452033e-01],
       [-3.65479483e-01],
       [-1.18588198e+00],
       [-4.98910412e-01],
       [-7.01477252e-01],
       [-3.34568761e-01],
       [-9.61141789e-01],
       [-5.90091584e-01],
       [-1.18705456e-01],
       [-1.88101881e-01],
       [-1.52981758e+00],
       [-1.59873342e-01],
       [-6.87387661e-01],
       [-1.89533627e+00],
       [-3.02700072e+00],
       [-1.42747003e-01],
       [ 2.71210697e-01],
       [-1.22087665e+00],
       [-6.32285424e-01],
       [-2.22549679e+00],
       [-1.17814733e+00],
       [-1.97296850e+00],
       [-4.34697209e-01],
       [-1.09728820e+00],
       [-6.96052262e-01],
       [-3.61098834e-01],
       [-2.26085022e+00],
       [-1.99850325e+00],
       [-2.5

Test_Train_Split

In [12]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(transformed_data, dataset['class_label'].values, test_size=0.2,random_state=42)


In [13]:

model = LogisticRegression()
model.fit(train_data, train_label)

LogisticRegression()

In [14]:
from sklearn.metrics import accuracy_score

final_predictions = model.predict(test_data)
for i in range(0,len(final_predictions)):
    if final_predictions[i] < 0.5:
        final_predictions[i] = 0
    else:
        final_predictions[i] = 1
        
accuracy = accuracy_score(test_label, final_predictions)
accuracy

0.8833333333333333

# Applying PCA then FDA


In [15]:
labels.info()

<class 'pandas.core.series.Series'>
Int64Index: 297 entries, 0 to 301
Series name: AHD
Non-Null Count  Dtype
--------------  -----
297 non-null    int64
dtypes: int64(1)
memory usage: 4.6 KB


In [16]:
from sklearn.decomposition import PCA

pca = PCA(0.99)
X_PCA = pca.fit_transform(dataset)
X_PCA.shape
X_PCA = np.insert(X_PCA, X_PCA.shape[1], labels.values,axis=1)

In [17]:
pca.explained_variance_ratio_

array([0.74693362, 0.14984584, 0.08586582, 0.01601508])

In [18]:
dataframe_ = pd.DataFrame(X_PCA)

In [19]:
dataframe_ = dataframe_.rename(columns={4:'class_label'})
dataframe_

,0,1,2,3,class_label
0,-13.324180,2.931010,14.984466,6.689517,0.0
1,40.575088,45.622868,21.377886,0.006701,1.0
2,-18.402425,21.358778,-11.748963,11.064696,1.0
3,1.830524,-39.890574,1.244383,-10.801756,0.0
4,-43.893181,-23.913942,2.153175,-8.037886,0.0
...,...,...,...,...,...
292,-5.771699,27.452873,5.053695,-3.075238,1.0
293,15.178632,12.713481,-25.858291,-9.408890,1.0
294,-53.090305,13.123448,15.565375,11.400841,1.0
295,-116.045876,35.748435,0.172438,0.249057,1.0


In [20]:
dataframe_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   0            297 non-null    float64
 1   1            297 non-null    float64
 2   2            297 non-null    float64
 3   3            297 non-null    float64
 4   class_label  297 non-null    float64
dtypes: float64(5)
memory usage: 11.7 KB


In [21]:
preModelFDA = FDA(dataframe_)
transformed_Data = preModelFDA.fit_transform()

In [22]:
transformed_Data.shape

(297, 1)

Test Train Split

In [23]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(transformed_Data, labels.values, test_size=0.2,random_state=42)


In [24]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_data, train_label)

LogisticRegression()

In [25]:
from sklearn.metrics import accuracy_score

final_predictions = model.predict(test_data)
for i in range(0,len(final_predictions)):
    if final_predictions[i] <= 0.5:
        final_predictions[i] = 0
    else:
        final_predictions[i] = 1
        
accuracy = accuracy_score(test_label, final_predictions)
accuracy

0.7166666666666667